# Initialisation

/!\ FIRST THING TO DO :

EXECUTION --> MODIFIER LE TYPE D'EXECUTION --> GPU (Accélérateur matériel)

In [55]:
# Install TensorFlow 2.0 on Google Collab if needed
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass

Verify GPU usage. You should see 

```
Number of GPUs available : 1
Only GPU number 0 used
```

In [ ]:
num_GPU = 0

import tensorflow as tf
print('Tensorflow version: ', tf.__version__)

gpus = tf.config.experimental.list_physical_devices("GPU")
print('Number of GPUs available :', len(gpus))

tf.config.experimental.set_visible_devices(gpus[num_GPU], 'GPU')
tf.config.experimental.set_memory_growth(gpus[num_GPU], True)
print('Only GPU number', num_GPU, 'used')

In [56]:
import numpy as np
import matplotlib.pyplot as plt  

from tensorflow.keras import Model
from tensorflow.keras.layers import Layer

### Intro

In this project, the goal is to create a set of NN-based transmitter and receiver that beats a conventional QAM system. 

The performance metric is the rate of the transmission. The rate of a transmission is the maximum number of bits that could be sent through the transmitter-receiver if perfect channel coding was used (the bits are coded/decoded before/after the transmissions). In this project, we won't use channel coding, so remember that the rate is a good indication of the performance of a system.

The questions should be answered in the report. Please contact me when you think you have the answer so that I can follow your progress :)

The goal is to finish section 1 and 2. Section 3 is optional. 

### Usefull functions
You do not have to use them all!

- https://www.tensorflow.org/api_docs/python/tf/cast?hl=fr --> tensorflow operations can only be carried out on tensors that have the same type (int8, float32, complex64...)
- https://www.tensorflow.org/api_docs/python/tf/shape
- https://www.tensorflow.org/api_docs/python/tf/size?hl=fr
- https://www.tensorflow.org/api_docs/python/tf/reshape
- https://www.tensorflow.org/api_docs/python/tf/concat
- https://www.tensorflow.org/api_docs/python/tf/one_hot
- https://www.tensorflow.org/api_docs/python/tf/math/reduce_mean
- https://www.tensorflow.org/api_docs/python/tf/math/reduce_sum
- https://www.tensorflow.org/api_docs/python/tf/expand_dims
- https://www.tensorflow.org/api_docs/python/tf/linalg/matmul
- https://www.tensorflow.org/api_docs/python/tf/size?hl=fr

### Usefull jupyter notebook commands

- `Ctrl + Enter` : Execute command
- `Shift + Enter` : Execute command and go to next cell
-  `dd` : Delete cell
- 'b' : Create new cell below
- 'a' : Create new cell above
- The little arrow next to titles : group cells from the specific section

# Predefined functions

## Not intersting

Do not look at this code, it is boring

In [ ]:
def visualize (model, SNRs_db):
    K = model.K
    msg = np.arange(2**K)
    res = []
    for i in range(2**K):
        temp=msg[i]
        res_ = []
        for j in 2**np.linspace(K-1, 0, K):
            if temp - j >= 0 :
                res_.append(1)
                temp -= j
            else:
                res_.append(0)
        res.append(res_)
    bits = np.array(res, dtype=np.float32) 
    
    fig, axes = plt.subplots(1, len(SNRs_db), figsize=(len(SNRs_db)*7, 6))
    if len(SNRs_db) == 1 :
        axes = np.array([axes])
    for i in range(len(SNRs_db)):
                   
        llrs = model(bits, tf.ones(shape=(tf.shape(bits)[0], 1))*SNRs_db[i])
        x = model.x
        if tf.size(tf.shape(x)) == 1 :
            x = x[:, tf.newaxis]
                   
        axes[i].set_title('SNR_db = '+str(SNRs_db[i]))
        for j in range(2**K):
            axes[i].scatter(np.real(x[j, 0]), np.imag(x[j, 0]))
            axes[i].text(np.real(x[j, 0])+0.02, np.imag(x[j, 0])+0.02, np.binary_repr(j, width=K), fontsize=8)
            axes[i].set_xlim([-1.65, 1.65])
            axes[i].set_ylim([-1.65, 1.65])

In [ ]:
def visualize_channel_uses (model, SNR_db):
    K = model.K
    msg = np.arange(2**K)
    res = []
    for i in range(2**K):
        temp=msg[i]
        res_ = []
        for j in 2**np.linspace(K-1, 0, K):
            if temp - j >= 0 :
                res_.append(1)
                temp -= j
            else:
                res_.append(0)
        res.append(res_)
    bits = np.array(res, dtype=np.float32) 
    llrs = model(bits, tf.ones(shape=(tf.shape(bits)[0], 1))*SNR_db)
    
    x = model.x
    
    if tf.size(tf.shape(x)) == 1 :
        x = x[:, tf.newaxis]
    
    fig, axes = plt.subplots(1, model.Nc, figsize=(model.Nc*7, 6))
    if model.Nc == 1 :
        axes = np.array([axes])
    for i in range(model.Nc):
        if model.Nc != 1 :
            axes[i].set_title('Channel use n° '+str(i+1))
        for j in range(2**K):
            axes[i].scatter(np.real(x[j, i]), np.imag(x[j, i]))
            axes[i].text(np.real(x[j, i])+.02, np.imag(x[j, i])+.02, np.binary_repr(j, width=K), fontsize=8)
            axes[i].set_xlim([-1.65, 1.65])
            axes[i].set_ylim([-1.65, 1.65])

In [ ]:
def table_gray(K):
        def genCode(n):
                if n == 0:
                    return ['']

                code1 = genCode(n-1)
                code2 = []
                for codeWord in code1:
                    code2 = [codeWord] + code2

                for i in range(len(code1)):
                    code1[i] += '0'
                for i in range(len(code2)):
                    code2[i] += '1'
                return code1 + code2  

        k = K // 2 # bits per dimension
        syms = 2**k
        m = syms**2
        # gray labeling
        gray_labeling = []
        for i in range(2**k):
            gray_labeling.append(np.array([int(e) for e in  genCode(k)[i]]))
        gray_labeling = np.array(gray_labeling)

        encoding_table_256_qam = gray_labeling[:,::-1]

        encoding_table_256_qam = np.hstack( [np.kron(encoding_table_256_qam,np.ones((syms,1))), np.tile(encoding_table_256_qam,(syms,1)) ])

        natural_number = np.sum(2**np.arange(2*k)[::-1] * encoding_table_256_qam,axis=1)
        table_gray = np.zeros((m,m))
        for bvec in range(m):
            table_gray[bvec,np.where(natural_number==bvec)[0]] =1

        return table_gray

In [ ]:
def qam_const(K):
    M = 2**K
    size = int(np.sqrt(M))
    a = np.arange(size)
    b = a - np.mean(a)
    C=[]
    for i in range(size):
        for j in range(size):
            C.append(b[j]-1j*b[i])
    C = C/np.sqrt(np.mean(np.square(np.abs(C))))
    return C.astype(np.complex64)


In [ ]:
def evaluate_rate(model, SNRs_eval, DS_size=10000):
    
    print('K = '+str(model.K))
    
    DS_bits, DS_SNRs = gen_DS(DS_size, model.K, 1., 1.)
    TF_DS_test = tf.data.Dataset.from_tensor_slices((DS_bits, DS_SNRs)).shuffle(DS_size).batch(1000)
    
    print("SNR (dB) = ", end='')
    rates = []
    for snr_i in SNRs_eval :

        rates_temp = []
        for step, (bits, SNRs) in enumerate(TF_DS_test):
            pb_eq_1 = model(bits, SNRs*snr_i)
            bits_pred = tf.sign(pb_eq_1-0.5)/2.+0.5
            err_1 = tf.abs(bits - bits_pred)
            err_2 = tf.abs(bits_pred - bits)
            err_3 = tf.reduce_sum(tf.nn.relu(err_1 - err_2)) * tf.reduce_mean(tf.nn.tanh(err_2 - err_1))
            err_4 =  tf.reduce_mean(tf.keras.losses.binary_crossentropy(bits, pb_eq_1)) * (tf.reduce_max(bits) + err_3)
            rates_temp.append(model.K-err_4.numpy())
        rates.append(np.mean(rates_temp))
        print(str(snr_i)+', ', end='')
    print('')
    return rates

## Could be interesting

This could be interesting

In [ ]:
def gen_DS(DS_size, K, SNR_db_low, SNR_db_high):
    """Returns a dataset of dimension [epochs, Bs, K]"""
        
    DS_bits = tf.random.uniform(shape=[DS_size, K], minval=0, maxval=2, dtype=tf.int32)
    DS_SNRs = tf.random.uniform(shape=[DS_size, 1], minval=SNR_db_low, maxval=SNR_db_high)
    return tf.cast(DS_bits, tf.float32), DS_SNRs

# Predefined classes

## Not interesting

Boring or "complex" stuff

In [ ]:
class Mapper(Layer):
    """
    Maps vector of K bits to a complex symbol
    Inputs :
    - K, the nber of bits/ch_uses
    """
    def __init__(self, K, Nc, rayleigh):
        super().__init__()     
        
        self.K = K
        self.Nc = Nc
        self.rayleigh = rayleigh
        
        if self.rayleigh == True :
            self.Nc -= 1
            if self.Nc < 1:
                print('/!\ with Rayleigh fading, Nc must be higher than one (Nc >= 2)')
                raise Exception()
        self.bin_conv = tf.convert_to_tensor(np.flip([np.power(2, i) for i in range(self.K)]), dtype=tf.float32)
        
    def call(self, bits, GM, QAM_const):
        """
        Inputs:
        - bits = [Bs, K] 
        - GM, the Gray Mapping
        - C = [M], the corresponding constallation
        
        Outputs:
        - x = [Bs, Nc] 
        """
        self.bits = bits
        self.bits_rshp = tf.reshape(bits, [-1, tf.shape(bits)[-1]])
        
        self.C_batch = tf.tile(tf.expand_dims(QAM_const, axis=0), [tf.shape(bits)[0], 1])
        
        self.s_dec = tf.cast(tf.reduce_sum(self.bits_rshp*self.bin_conv[tf.newaxis, :], axis=1), tf.int32) # [BS]
        self.s_idx = tf.gather(GM, self.s_dec, axis=0)
        self.s_idx = tf.argmax(self.s_idx, axis=1, output_type=tf.int32)
        
        self.s_oh = tf.one_hot(self.s_idx, depth=tf.pow(2, self.K))
        self.C_exp = tf.reshape(tf.tile(self.C_batch[:, tf.newaxis, :], [1, 1, 1]), [-1, tf.pow(2, self.K)])[:, :, tf.newaxis]
        
        # Mapping symbols to complex points
        self.x_rshp = tf.squeeze(tf.matmul(tf.cast(self.s_oh[:, tf.newaxis, :], tf.complex64), self.C_exp))
        self.x = tf.tile(tf.reshape(self.x_rshp, [tf.shape(bits)[0], 1])[:, tf.newaxis, :], [1, self.Nc, 1])[:, :, 0]
        
        if self.rayleigh == True:
            self.x = tf.concat([self.x, tf.ones([tf.shape(self.x)[0], 1], dtype=tf.complex64)], axis=1)
            
        return self.x

In [ ]:
class Channel(Layer):

    def __init__(self, Nc=1, rayleigh=False, **kwargs):
        super(Channel, self).__init__(**kwargs)
        
        self.Nc = Nc
        self.rayleigh = rayleigh
        
    def call(self, x, SNR_db):
        
        if tf.reduce_mean(tf.square(tf.abs(x)))>2.5:
            tf.print('Watch out! You mean power should be one, but is actually ' + str(tf.reduce_mean(tf.square(tf.abs(x))).numpy()))
            raise Exception ('Stop due to wrong average symbol power')
        
        dims_1 = False
        if tf.size(tf.shape(x)) == 1 :
            x = x[tf.newaxis]
            dims_1 = True
            
        if self.rayleigh == False:
            noise_std = tf.sqrt( 1 / tf.pow(10., SNR_db/10.0))
            noise_r = tf.random.normal(shape = [tf.shape(x)[0], self.Nc], stddev = 1) * noise_std/tf.sqrt(2.)
            noise_i = tf.random.normal(shape = [tf.shape(x)[0], self.Nc], stddev = 1) * noise_std/tf.sqrt(2.)
            self.noise_cplx = tf.complex(noise_r, noise_i)
        
            self.y = x + self.noise_cplx
        
        else:
            rayleigh_real = tf.random.normal(shape = [tf.shape(x)[0]], stddev = 1./tf.sqrt(2.))
            rayleigh_imag = tf.random.normal(shape = [tf.shape(x)[0]], stddev = 1./tf.sqrt(2.))
            self.h = tf.tile(tf.complex(rayleigh_real, rayleigh_imag)[:, tf.newaxis], [1, self.Nc])

            noise_std = tf.sqrt( 1 / tf.pow(10., SNR_db/10.0))
            noise_r = tf.random.normal(shape = [tf.shape(x)[0], self.Nc], stddev = 1) * noise_std/tf.sqrt(2.)
            noise_i = tf.random.normal(shape = [tf.shape(x)[0], self.Nc], stddev = 1) * noise_std/tf.sqrt(2.)
            self.noise_cplx = tf.complex(noise_r, noise_i)

            self.y = self.h*x + self.noise_cplx
        
        if dims_1 == True :
            self.y = self.y[:, 0]
            
        return self.y

In [ ]:
class Demapper(Layer):

    def __init__(self, K, GM, rayleigh, **kwargs):
        super().__init__(**kwargs)
        
        self.K = K
        self.const_size = 2**K
        self.rayleigh = rayleigh
        
        self.bits2symb = np.argmax(GM, axis=1)
        bit_list = np.arange(self.const_size)
        bit_list = [[int(j) for j in list(np.binary_repr(i, width=self.K))] for i in bit_list]
        self.bits_nat = np.array(bit_list)

        self.ind_all = np.arange(self.const_size, dtype=np.int64)

        self.B = []
        for i in range(self.K):
            self.B.append([None, None])
            # 0
            a = np.where(self.bits_nat[:,i] == 0)[0]
            self.B[i][0] = np.take(self.bits2symb, a)
            # 1
            a = np.where(self.bits_nat[:,i] == 1)[0]
            self.B[i][1] = np.take(self.bits2symb, a)
    
    
    def awgn_cde(self, snr_db, y, s, C):
        mu = tf.gather(C, s, axis=1)
        snr = tf.pow(tf.cast(10.0, tf.float32), snr_db/10.0)
        v = 1./(snr)
        z = -tf.reduce_sum(tf.square(y - mu), axis=2)/v
        
        return z
    
    def call(self, y_cplx, SNR_db, QAM_const):
        """
        AWGN demapper for SISO links
        
        Inputs : y_cplx = [Bs, Nc], snr_db = [1], C = [M]
        Outputs : LLRs = [Bs, K]
        """
        if self.rayleigh==True:
            y_cplx_eq = y_cplx[:, :-1] / y_cplx[:, -1:]
            y_cplx_mean = tf.reduce_mean(y_cplx_eq, axis=1, keepdims=True)
        else :
            y_cplx_mean = tf.reduce_mean(y_cplx, axis=1, keepdims=True)
            
        SNR_db_batch = SNR_db #tf.ones([tf.shape(y_cplx_mean)[0], 1]) * SNR_db
        C_batch = tf.tile(tf.expand_dims(QAM_const, axis=0), [tf.shape(y_cplx_mean)[0], 1])
        
        C_r_i = tf.stack([tf.math.real(C_batch), tf.math.imag(C_batch)], axis=2)
        y = tf.stack([tf.math.real(y_cplx_mean), tf.math.imag(y_cplx_mean)], axis=2) 
        
        pyb_0 = []
        pyb_1 = []
        y = tf.tile(y, [1, self.const_size//2, 1])
        y_2 = tf.tile(y, [1, 2, 1])
        p = tf.constant(self.ind_all, tf.int32)
        z_all = self.awgn_cde(SNR_db_batch, y_2, p, C_r_i)
        for i in range(self.K):
            # 0
            p = tf.constant(self.B[i][0], tf.int32)
            z = tf.gather(z_all, p, axis=1)
            z = tf.math.reduce_logsumexp(z, axis=1)
            pyb_0.append(z)
            # 1
            p = tf.constant(self.B[i][1], tf.int32)
            z = tf.gather(z_all, p, axis=1)
            z = tf.math.reduce_logsumexp(z, axis=1)
            pyb_1.append(z)
        pyb_0 = tf.stack(pyb_0, axis=1)
        pyb_1 = tf.stack(pyb_1, axis=1)
        llr = pyb_1 - pyb_0
        
        return llr

## Could be interesting

You might want to look at it

In [ ]:
class Model_QAM(Model) :
    def __init__(self, K, Nc=1, rayleigh=False):
        super().__init__()
        
        self.K = K
        self.Nc = Nc 
        self.QAM_const = tf.constant(qam_const(K))
        self.GM = tf.constant(table_gray(K), tf.float32)

        self.mapper = Mapper(self.K, self.Nc, rayleigh)
        self.channel = Channel(self.Nc, rayleigh)
        self.demapper = Demapper(self.K, self.GM, rayleigh)

    #@tf.function
    def call(self, bits, SNR_db):
        
        self.bits = tf.cast(bits, tf.float32)
        
        self.x = self.mapper(bits, self.GM, self.QAM_const)
            
        self.y = self.channel(self.x, SNR_db)

        self.llrs = self.demapper(self.y, SNR_db, self.QAM_const)
        
        self.pb_eq_1 = tf.nn.sigmoid(self.llrs)
        
        return self.pb_eq_1

# Baseline (QAM) system

<img src="https://github.com/mgoutay/ml_course/blob/master/Images/standard_system.png?raw=true" alt="Drawing" style="width: 700px;"/>

### QAM transmission

Let's start by defining the baseline that we want to beat : a 64-QAM transmitter/receiver

The number of bits represented by one complex symbol is denoted $K$

Using a 64-QAM transmitter means that we use one complex symbol to transmit 6 bits ($K=6$)

- choosing $K=2$ means that QPSK (= 4-QAM) is used
- choosing $K=4$ means that 16-QAM is used
- etc..

### Notations

(scalars are denoted by lower-case letters, vectors by bold lower-case letters)

The bits that we want to send over the channel are denoted $\mathbf{b}\in\{0, 1\}^{K}$. It is a vector of bits (0, 1) of size $K$.

The corresponding complex symbol sent by the transmitter is denoted $x\in\mathbb{C}$, and the symbol received by the receiver is denoted $y\in\mathbb{C}$

Finally, the receiver outputs probabilities $\mathbf{p}\in\mathbb{R}^K$. Each entry $p_i$ of $\mathbf{p}$ correspond to the probability that the $i^{\text{th}}$ bit equals one, so we have $0 \leq p_i \leq 1$

### Channel model

We consider a Additive White Gaussian Noise (AWGN) channel, where we have: 
$$y = x + n$$
$n$ being complex gaussian noise with variance $\sigma^2$ : $n \sim \mathcal{C}\mathcal{N}(0, \sigma^2) $, meaning that the variance of both the real and imaginary part is $\frac{\sigma^2}{2}$.

It is assumed that $\mathbb{E}[|x|^2] =1$, i.e., the average power per symbol is one.

The Signal to Noise Ratio (SNR) of the transmission is defined as $\text{SNR} = \frac{\mathbb{E}[|x|^2]}{\sigma^2} = \frac{1}{\sigma^2} $. It is common practice to represent the SNR in decibel : $\text{SNR}_{\text{dB}} = 10 \text{log}_{10} (\text{SNR})$

In [ ]:
K=6
DS_size=10000 #Size of the Dataset to generate

DS_bits, DS_SNRs_db = gen_DS(DS_size, K, SNR_db_low=-5, SNR_db_high=30)

### Dataset

The datasets are generated by the `gen_DS` function, which returns a dataset composed of random bits of dimension [DS_size, K] and random SNRs of dimension [DS_size, 1] in dB. 

In [ ]:
print(DS_bits.shape)
print(DS_SNRs_db.shape)

In [ ]:
print(DS_bits[:2]) #Display only the first two entries in the dataset
print(DS_SNRs_db[:2]) #Display only the first two entries in the dataset

We can now initialize our baseline model, and make a simple forward pass through it. The output of the model is the estimated probabilities that the received bits are ones. Let's try with the first two sample of our dataset :

In [ ]:
# Initialize the baseline
baseline = Model_QAM(K)

# Define a very small batch just just to visualize the output
batch_bits = DS_bits[:2]
batch_SNRs = DS_SNRs_db[:2]

# Print the sent bits
print(batch_bits)

# Compute probabilities that the bits equals to one
prob_eq_1 = baseline(batch_bits, batch_SNRs)

# Print the symbols that were sent
print(baseline.x)

# Print the predicted probabilities
print(prob_eq_1)

We can vizualize the symbol sent at different SNRs (and the associated bits) using the `visualize()` function : 

In [ ]:
visualize(baseline, SNRs_db=[0, 5, 10, 20])

Can you see something special about the way the symbols are attanged?

Finally, the rate of a transmission is the maximum number of bits that could be sent through the transmitter-receiver if perfect channel coding was used (the bits are coded/decoded before/after the transmissions). In this project, we won't use channel coding, so remember that the rate is a good indication of the performance of a system. The computation of the rate is a bit complex so do not try to undersand it ;)

We can evaluate the rate of the transmission at different SNRs using the `evaluate_rate()` function:

In [ ]:
SNRs_eval = np.linspace(0, 25, 6)
rates_baseline = evaluate_rate(baseline, SNRs_eval)

plt.plot(SNRs_eval, rates_baseline)
plt.xlabel('SNR')
plt.ylabel('Rate')
plt.grid()

These are the rates we want to beat! 

# 1- Get your hands dirty

<img src="https://github.com/mgoutay/ml_course/blob/master/Images/NN-based_system.png?raw=true" alt="Drawing" style="width: 700px;"/>

In this first section, the goal is to create your first autoencoder. To make trainings faster, we set $K = 4$ here. Do not try to beat the baseline, as it might be hard for $K<6$

The main objectives are to :
- Define a loss function
- Create the NN-based transmitter and receiver 
- To normalize the output so that the average power per symbol is one
- Create a training loop
- Train, evaluate, and compare your system to the baseline

You need to make sure that your average power per symbol is one !
You can compute your average energy by computing `tf.reduce_mean(tf.square(tf.abs(x)))` once you have performed at least one forward path (so that self.x  contains some values)

$$\mathbb{E}[|x|^2] \approx \frac{1}{B_s} \sum_{i=1}^{B_s} |x|^2 $$

How can you make sure that $\frac{1}{B_s} \sum_{i=1}^{B_s} |x|^2 = 1$ ?

### 1.0 - Import the layer(s) you need

In [ ]:
### Import the layer(s) you need

### 1.1 -  Define the model

In [ ]:
# This is your autoencoder model, comprising the transmitter, channel, and receiver
class Autoencoder(Model):

    def __init__(self, K, **kwargs):
        super(Autoencoder, self).__init__(**kwargs)
        
        self.K = K
        #You can set other attributes
        
        self.tx_layer_1 = ### Define transmitter layers
        self.tx_layer_2 = ### Define transmitter layers
        self.tx_layer_3 = ### Define transmitter layers
        ### ...
        
        self.channel = Channel() 
        
        self.rx_layer_1 = ### Define receiver layers
        self.rx_layer_2 = ### Define receiver layers
        self.rx_layer_3 = ### Define receiver layers
        ### ...
    
    def call(self, bits, SNR_db):
        
        ### Use layers and/or other Tensorflow functions
        self.x = # This must be the symbols that are sent
            
        self.y = self.channel(self.x, SNR_db)
        
        ### Use layers and/or other Tensorflow functions
        self.pb = # This must be the probabilities that the sent bits are ones

        return self.pb

### 1.2 -  Instantiate the model, the loss, and the optimizer

In [ ]:
K = 4


autoencoder = Autoencoder(K) # Instantiate an autoencoder
baseline = Model_QAM(K) # Instantiate a baseline with the same K for comparison

#Initialize optimizer and loss function
optimizer = # Instantiate an optimizer
loss_func = # Pick a loss function that makes sense for this problem, do not try to understand how the rate is computed! If you choose a good loss funciton, the rate of the transmission will be maximized

What loss function did you use and why?

### 1.3 - generate and create optimized Tensorflow datasets

In [ ]:
DS_size = 100000

batch_size = 1024
DS_bits, DS_SNRs_db = gen_DS(DS_size, K, SNR_db_low=-5, SNR_db_high=30)

TF_DS_train = # Create optimized TF dataset
TF_DS_test = # Create optimized TF dataset

### 1.4 - Train the model

In [ ]:
### Training loop 

### 1.5 - Evaluate the model

In Section 1, the goal is to have a model that works. You can evaluate the model with the `evaluate_rate()` function, but do not try to beat the baseline.

In [ ]:
# Define the SNRs (dB) at which we want to evaluate the models
SNRs_eval = np.linspace(0, 20, 5)

# Evaluate the models
rate_autoencoder = evaluate_rate(autoencoder, SNRs_eval)
rate_baseline = evaluate_rate(baseline, SNRs_eval) # For comparison

#plot the result
plt.plot(SNRs_eval, rate_baseline, label='Baseline')
plt.plot(SNRs_eval, rate_autoencoder, label='Autoencoder')
plt.xlabel('SNR')
plt.ylabel('Rate')
plt.legend()
plt.grid()

We can vizualize the learned constellations at different SNRs:

What do you see (two things should be noticed)?

In [ ]:
visualize(baseline, SNRs_db=[0, 5, 10, 20]) #Visualize the symbols generated by the conventional QAM system
visualize(autoencoder, SNRs_db=[0, 5, 10, 20]) #Visualize the symbols generated by the autoencoder

Send three batches of random bits and plot the sent symbols for a fixed SNR of 10dB:

What do you see ?

In [ ]:
SNRs =  tf.ones((100, 1))*10.

rdm_bits = tf.cast(tf.random.uniform([100, K], 0, 2, dtype=tf.int32), tf.float32)
pb = autoencoder(rdm_bits, SNRs)
plt.scatter(np.real(autoencoder.x), np.imag(autoencoder.x), marker='+', label='1st transmission')

rdm_bits = tf.cast(tf.random.uniform([100, K], 0, 2, dtype=tf.int32), tf.float32)
pb = autoencoder(rdm_bits, SNRs)
plt.scatter(np.real(autoencoder.x), np.imag(autoencoder.x), marker='+', label='2nd transmission')

rdm_bits = tf.cast(tf.random.uniform([100, K], 0, 2, dtype=tf.int32), tf.float32)
pb = autoencoder(rdm_bits, SNRs)
plt.scatter(np.real(autoencoder.x), np.imag(autoencoder.x), marker='+', label='3rd transmission')

plt.legend()

# 2. Generate a proper constellation

Here we want to solve the problem that we have spotted just above. Please call me before starting Section 2 so that I can make sure that you understand the problem.

The idea here is to use the transmitter to generate constellation vectors that depends on the SNR, instead of directly generating the symbols to send. For one entry in the batch (one SNR), this constellation vector contains all $2^K$ symbols that can be sent, so it is a complex vector of size $2^K$.

To generate the sent symbols, one can then:
- For each entries in the batch, convert the vector of bits to be send to their one-hot representation. For example, with $K=2$ : [00] = [1000], [10] = [0100], [01] = [0010], [11] = [0001] or something similar.  You might want to use `tf.pow()` and `tf.one_hot()`
- Multiply (element-wise) the one-hot representation of the bits to the corresponding constellation vector generated by the transmitter
- The resulting vector is of size $2^K$ but only one entry is non-zero. This can be simplified to a single complex number, for example using `tf.reduce_sum()` on the good axis

What is the benefit of using this technique?

### 2.1 Define the model

In [ ]:
class Autoencoder2(Model):

    def __init__(self, K, **kwargs):
        super(Autoencoder2, self).__init__(**kwargs)
        
        self.K = K
        #You can set other attributes
        
        self.tx_layer_1 = ### Define transmitter layers
        self.tx_layer_2 = ### Define transmitter layers
        self.tx_layer_3 = ### Define transmitter layers

        self.channel = Channel()
        
        self.rx_layer_1 = ### Define receiver layers
        self.rx_layer_2 = ### Define receiver layers
        self.rx_layer_3 = ### Define receiver layers
    
    def call(self, bits, SNR_db):
        
        ### Use layers and/or other Tensorflow functions
        self.x = # This must be the symbols that are sent
            
        self.y = self.channel(self.x, SNR_db)
        
        ### Use layers and/or other Tensorflow functions
        self.pb = # This must be the probabilities that the sent bits are ones

        return self.pb

You can start by setting $K=4$, but remember to use $K=6$ when you try to beat the baseline

### 2.2 -  Instantiate the model, the loss, and the optimizer

In [ ]:
K = 6

# Instantiate an autoencoder
autoencoder2 = Autoencoder2(K)
baseline2 = Model_QAM(K)

#Initialize optimizer and loss function
optimizer = # Optimizer
loss_func = # Loss function

### 2.3 - generate and create optimized Tensorflow datasets

In [ ]:
DS_size = 100000
batch_size = 1024

DS_bits, DS_SNRs_db = gen_DS(DS_size, K, SNR_db_low=-5, SNR_db_high=25)
print('Dataset generated with K =', K)

TF_DS_train = # Create optimized TF dataset
TF_DS_test = # Create optimized TF dataset

### 2.4 - Train the model


In [ ]:
### training loop

### 2.5 - Evaluate the model

Do you manage to beat the baseline with $K=6$ ?

In [ ]:
SNRs_eval = np.linspace(0, 20, 5)
rate_autoencoder2 = evaluate_rate(autoencoder2, SNRs_eval)
rate_baseline2 = evaluate_rate(baseline2, SNRs_eval)

plt.plot(SNRs_eval, rate_baseline2, label='Baseline2')
plt.plot(SNRs_eval, rate_autoencoder2, label='Autoencoder2')
plt.xlabel('SNR')
plt.ylabel('Rate')
plt.legend()
plt.grid()

We can also visualize the sent constellation at different SNRs : 

In [ ]:
visualize(autoencoder2, SNRs_db=[0, 5, 10, 20])
visualize(baseline2, SNRs_db=[0, 5, 10, 20])

# 3 - Transmit on multiple channel uses

One vector of $K$ bits can also be transmitted on multiple channel uses, meaning that for each vector of $K$ bits in the batch we send $N_c$ complex symbols instead of $1$. $N_c$ denotes the number of channel uses, and the symbols sent that correspond to one vector of bits are denoted $\mathbf{x}\in \mathbb{C}^{N_c}$

The goal here is to transmit $N_c$ symbols for each entry in the batch, making sure that the average power per symbol remains one : $\mathbb{E}[[x|] =1$.

### 3.0 - Example with the baseline, with $N_c=2$

In [ ]:
K=6
Nc = 2

DS_size=100 
DS_bits, DS_SNRs_db = gen_DS(DS_size, K, SNR_db_low=-5, SNR_db_high=30) #Generate a dataset

# Initialize the baseline
baseline3 = Model_QAM(K, Nc)

# Define a very small batch containing just one sample to visualize the output
batch_bits = DS_bits[:1]
batch_SNRs = DS_SNRs_db[:1]

# Print the sent bits
print('batch_bits : ', batch_bits)
print('batch_SNRs : ', batch_SNRs)

# Compute probabilities that the bits equals to one
prob_eq_1 = baseline3(batch_bits, batch_SNRs)

# Print the symbols that were sent
print('baseline.x : ', baseline3.x) # x is now a vector of size Nc !

# Print the predicted probabilities
print('pb predicted by the baseline : ', prob_eq_1)

You can visualize the $N_c$ channel uses with the `visualize_channel_uses()` function

In [ ]:
visualize_channel_uses(baseline3, SNR_db = 10)

### 3.1 - Build the autoencoder, train, and evaluate

In [ ]:
class Autoencoder3(Model):

    def __init__(self, K, Nc, **kwargs):
        super(Autoencoder3, self).__init__(**kwargs)
        
        self.K = K
        self.Nc = Nc
        #You can set other attributes
        
        self.tx_layer_1 = ### Define transmitter layers
        self.tx_layer_2 = ### Define transmitter layers
        self.tx_layer_3 = ### Define transmitter layers

        self.channel = Channel(self.Nc)
        
        self.rx_layer_1 = ### Define receiver layers
        self.rx_layer_2 = ### Define receiver layers
        self.rx_layer_3 = ### Define receiver layers
    
    def call(self, bits, SNR_db):
        
        ### Use layers and/or other Tensorflow functions
        self.x = # This must be the symbols that are sent
            
        self.y = self.channel(self.x, SNR_db)
        
        ### Use layers and/or other Tensorflow functions
        self.pb = # This must be the probabilities that the sent bits are ones

        return self.pb

In [ ]:
K = 6
Nc = 2

# Instantiate an autoencoder
autoencoder3 = Autoencoder3(K, Nc)
baseline3 = Model_QAM(K, Nc)

#Initialize optimizer and loss function
optimizer =  ### Optimizer
loss_func = ### Loss function

In [ ]:
DS_size = 100000
batch_size = 1024
DS_bits, DS_SNRs_db = gen_DS(DS_size, K, SNR_db_low=-5, SNR_db_high=25)
print('Dataset generated with K =', K)

TF_DS_train = # Create optimized TF dataset
TF_DS_test = # Create optimized TF dataset

In [ ]:
### Training loop

In [ ]:
SNRs_eval = np.linspace(0, 20, 5)
rate_autoencoder3 = evaluate_rate(autoencoder3, SNRs_eval)
rate_baseline3 = evaluate_rate(baseline3, SNRs_eval)

plt.plot(SNRs_eval, rate_baseline3, label='Baseline3')
plt.plot(SNRs_eval, rate_autoencoder3, label='Autoencoder3')
plt.xlabel('SNR')
plt.ylabel('Rate')
plt.legend()
plt.grid()

In [ ]:
visualize_channel_uses(autoencoder3, SNR_db = 10)

Can you see something interesting?